In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import re
import os

In [2]:
data = pd.read_csv("artist_data/artist_data_new.csv")
url_list = data["Wikipedia"]
url_name = data["url_name"]

In [12]:
def scrape_wikipedia(url, html_filename, text_filename):
    headers = {"User-Agent": "Mozilla/5.0"}  # User-Agent to prevent blocking
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Save full HTML
        with open(html_filename, "w", encoding="utf-8") as file:
            file.write(response.text)

        # Parse HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract main content
        content_div = soup.find("div", {"id": "mw-content-text"})
        if content_div:
            paragraphs = content_div.find_all("p")  # Extract only paragraphs
            raw_text = "\n".join([p.get_text() for p in paragraphs])

            # Save extracted text
            with open(text_filename, "w", encoding="utf-8") as text_file:
                text_file.write(raw_text)

    else:
        print(f"Failed to retrieve the page: {url}. Status code: {response.status_code}")

In [16]:
url

'http://mironov-art.com/'

In [14]:
folderlist = os.listdir("artist_wikipedia/")

for n in tqdm(range(len(url_list))):
    url = url_list[n]
    
    if str(url) != "nan":
        if "en/" in url_name[n]:
            screen_name = re.sub("en/", "", url_name[n])
        else:
            screen_name = url_name[n]
        
        if screen_name+".html" not in folderlist:
            try:
                scrape_wikipedia(url, f"artist_wikipedia/{screen_name}.html", f"artist_wikipedia_content/{screen_name}.txt")
            except ConnectionError as e:
                print(f"{e}: Wrong URL {url}")

  7%|▋         | 216/3048 [00:00<00:02, 949.39it/s]

Failed to retrieve the page: https://en.wikipedia.org/wiki/Artemisia_Gentileschi,. Status code: 404


 45%|████▌     | 1382/3048 [12:11<14:41,  1.89it/s]


ConnectionError: HTTPConnectionPool(host='mironov-art.com', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x10dcf1660>: Failed to resolve 'mironov-art.com' ([Errno 8] nodename nor servname provided, or not known)"))